# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,51.93,86,41,4.94,AU,1734193000
1,1,lima,-12.0432,-77.0282,70.05,68,75,5.75,PE,1734192928
2,2,moussoro,13.6408,16.4901,84.81,10,18,16.33,TD,1734193003
3,3,ilulissat,69.2167,-51.1000,19.42,68,40,8.05,GL,1734193004
4,4,waitangi,-43.9535,-176.5597,61.83,95,97,5.99,NZ,1734193007


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    line_color="black",
    color="City"
)
   
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
#Filter the DataFrame to find the cities that fit the criteria
print(f"I've always wanted to visit Australia.")


I've always wanted to visit Australia and New Zealand, so I'm looking for cities in those countries with a temperature between 70 and 80 degrees Fahrenheit, no cloudiness, and a wind speed less than 10 mph.


In [19]:
# Create a filter for Australian cities
aus_cities = ["AU"]

# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df["Country"].isin(aus_cities))
    & (city_data_df["Max Temp"] > 48)
    & (city_data_df["Max Temp"] < 87)
    & (city_data_df["Cloudiness"] > 0)
    & (city_data_df["Wind Speed"] > 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,51.93,86,41,4.94,AU,1734193000
24,24,perth,-31.9333,115.8333,68.67,80,75,10.36,AU,1734192901
78,78,margaret river,-33.9500,115.0667,62.96,82,13,12.03,AU,1734193091
83,83,codrington,-38.2667,141.9667,49.69,93,1,7.63,AU,1734193098
84,84,carnarvon,-24.8667,113.6333,76.42,78,14,17.07,AU,1734193099
95,95,karratha,-20.7377,116.8463,84.38,64,99,10.83,AU,1734193111
115,115,new norfolk,-42.7826,147.0587,54.72,97,11,2.57,AU,1734193134
139,139,port lincoln,-34.7333,135.8667,66.47,76,8,8.72,AU,1734193160
234,234,geraldton,-28.7667,114.6000,70.21,78,100,10.36,AU,1734193271
255,255,gunnedah,-30.9833,150.2500,76.48,50,2,5.73,AU,1734193294


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,86,
24,perth,AU,-31.9333,115.8333,80,
78,margaret river,AU,-33.9500,115.0667,82,
83,codrington,AU,-38.2667,141.9667,93,
84,carnarvon,AU,-24.8667,113.6333,78,
95,karratha,AU,-20.7377,116.8463,64,
115,new norfolk,AU,-42.7826,147.0587,97,
139,port lincoln,AU,-34.7333,135.8667,76,
234,geraldton,AU,-28.7667,114.6000,78,
255,gunnedah,AU,-30.9833,150.2500,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
#Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    response = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
perth - nearest hotel: Oxford Hotel
margaret river - nearest hotel: Margaret River Hotel
codrington - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
new norfolk - nearest hotel: Woodbridge on the Derwent
port lincoln - nearest hotel: Boston Hotel
geraldton - nearest hotel: Ocean Centre Hotel
gunnedah - nearest hotel: The Court House
charters towers - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
darwin - nearest hotel: Mantra Pantanas Darwin
swan hill - nearest hotel: Burke and Wills Motor Inn
sydney - nearest hotel: The Fullerton Hotel
andergrove - nearest hotel: No hotel found
moranbah - nearest hotel: No hotel found
mildura - nearest hotel: Chaffey Motor Inn
port augusta - nearest hotel: Hotel Commonwealth
port hedland - nearest hotel: The Esplanade Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,86,Villa Howden
24,perth,AU,-31.9333,115.8333,80,Oxford Hotel
78,margaret river,AU,-33.9500,115.0667,82,Margaret River Hotel
83,codrington,AU,-38.2667,141.9667,93,No hotel found
84,carnarvon,AU,-24.8667,113.6333,78,No hotel found
95,karratha,AU,-20.7377,116.8463,64,Karratha International Hotel
115,new norfolk,AU,-42.7826,147.0587,97,Woodbridge on the Derwent
139,port lincoln,AU,-34.7333,135.8667,76,Boston Hotel
234,geraldton,AU,-28.7667,114.6000,78,Ocean Centre Hotel
255,gunnedah,AU,-30.9833,150.2500,50,The Court House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    line_color="black",
    color="Hotel Name"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)